<a href="https://colab.research.google.com/github/LeeSeungwon89/Lecture-and-self-study/blob/master/5-3%20%ED%8A%B8%EB%A6%AC%EC%9D%98%20%EC%95%99%EC%83%81%EB%B8%94(%EC%9E%91%EC%97%85%20%EC%A4%91).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 트리의 앙상블

## 정형 데이터와 비정형 데이터

- 정형 데이터: 구조화 된 데이터를 의미하며 CSV, Database, Excel에 저장하기 쉬움. '앙상블 학습(Ensemble learning)' 은 정형 데이터를 다루는 데 가장 뛰어난 성과를 내는 결정 트리 기반 알고리즘임.

- 비정형 데이터: 비구조화 된 데이터로 텍스트 데이터, 사진, 디지털 음악 등을 의미함. 규칙성을 찾기 어려워 전통적인 머신러닝 방법으로 모델을 만들기 어려우므로 '신경망 알고리즘' 을 활용하여 모델을 만듦.

## 데이터 준비

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()

target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42)

## 랜덤 포레스트

- '랜덤 포레스트(Random Forest)' 는 앙상블 학습의 대표 주자로 안정적인 성능을 내며, 앙상블 학습을 적용할 때 가장 먼저 시도하길 권장하는 모델임. 결정 트리를 랜덤하게 만들어 결정 트리의 숲을 만들고, 각 결정 트리의 예측을 사용하여 최종 예측을 만듦. 

- 각 트리를 훈련하기 위한 데이터를 랜덤하게 만드는데, 입력한 훈련 데이터에서 랜덤하게 샘플을 추출하여 훈련 데이터를 만듦. 이때 한 샘플이 중복되어 추출될 수 있음.

 - e.g. 가방 1,000개에서 샘플을 100개씩 뽑는다면 먼저 1개를 뽑고, 뽑았던 1개를 다시 가방에 넣음. 이렇게 계속 100개를 가방에서 뽑으면 중복된 샘플을 뽑을 수 있는데 이 샘플을 '부트스트랩 샘플(Bootstrap sample)' 이라고 부름. 기본적으로 훈련 세트의 크기와 같게 만듦. 가방 1,000개에서 중복하여 샘플 1,000개를 뽑기 때문에 부트스트랩 샘플은 훈련 세트와 크기가 같음.

 - 부트스트랩(Bootstrap): 보통 부트스트랩 방식이라고 부르며, 위 예시처럼 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식을 의미함.

- 각 노드를 분할할 때 전체 특성 중에서 일부 특성을 무작위로 고른 다음 이 중에서 최선의 분할을 찾음.

 - RandomForestClassifier: 분류 클래스이며, 전체 특성 개수의 제곱근만큼 특성을 사용함. 즉 특성 4개가 있다면 노드마다 2개를 랜덤하게 선택하여 사용함.
 
 - RandomForestRegressor: 회귀 클래스이며, 전체 특성을 사용함.

- 기본적으로 결정 트리 100개를 이 방식으로 훈련함.

 - 분류일 경우에는 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측으로 삼음. 
 
 - 회귀일 경우에는 각 트리의 예측을 평균한 값을 예측으로 삼음.

- 랜덤하게 선택한 샘플과 특성을 사용하기 때문에 훈련 세트에 과대적합되는 것을 방지하고, 검증 세트와 테스트 세트에서 안정적인 성능을 얻을 수 있음. 종종 기본 매개변수 설정만으로도 좋은 결과를 냄.

- 결정 트리의 앙상블이므로 'DecisionTreeClassifier' 가 제공하는 중요한 매개변수인 'criterion', 'max_depth', 'max_feature', 'min_samples_split', 'min_impurity_decrease', 'min_samples_leaf' 등을 모두 제공함.

- 결정 트리의 큰 장점 중 하나인 특성 중요도를 계산함. 랜덤 포레스트의 특성 중요도는 각 결정 트리의 특성 중요도를 취합한 것임.

### 교차 검증 수행하기

In [2]:
# 교차 검증을 수행함.
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs = -1, random_state = 42)

scores = cross_validate(rf, train_input, train_target, return_train_score = True, n_jobs = -1)
print(scores)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 과대적합 상태임.
# 알고리즘을 살펴보는 것이 목적이므로 매개변수를 더 조정하지는 않음.
# 아울러 이 예제는 매우 간단하고 특성이 많지 않아서
# 그리드 서치를 사용하더라도 하이퍼파라미터 튜닝 결과가 크게 나아지지 않음.

{'fit_time': array([0.58024049, 0.58865094, 0.56386948, 0.57853699, 0.37614799]), 'score_time': array([0.10840678, 0.10218883, 0.10448742, 0.10225773, 0.10222578]), 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]), 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}
0.9973541965122431 0.8905151032797809


### 모델 훈련 후 특성 중요도 출력하기

In [3]:
# 모델 훈련 후 특성 중요도를 출력함. 특성 중요도는 'feature_importances_' 속성에 저장됨.
rf.fit(train_input, train_target)
print(rf.feature_importances_)
# 앞의 1절에서 결정 트리 모델의 특성 중요도는 [0.12345626, 0.86862934, 0.0079144] 이고, 각각 alcohol, sugar, pH를 나타냄. 
# 랜덤 포레스트 점수와 비교하면 alcohol과 pH의 중요도가 상승하고 sugar의 중요도가 감소함. 
# 이런 이유는 랜덤 포레스트가 특성의 일부를 랜덤하게 선택하여 결정 트리를 훈련하기 때문임.
# 즉 하나의 특성에 과도하게 집중하지 않고 좀 더 많은 특성이 훈련에 기여할 기회를 부여하는 것임.
# 이렇게 하여 과대적합을 줄이고 일반화 성능을 높이는 데 도움이 됨.

[0.23167441 0.50039841 0.26792718]


### OOB 샘플로 모델 평가하기

- 'RandomForestClassifier' 의 기능 중에 자체적으로 모델을 평가하는 점수를 얻을 수 있는 기능이 있음. 부트스트랩 샘플에 포함되지 않고 남는 샘플을 'OOB(out of bag)' 이라고 하는데, 이 남는 샘플을 사용하여 부트스트랩 샘플로 훈련한 결정 트리를 평가함. '검증 세트' 역할을 한다고 볼 수 있으며, 교차 검증을 대신하므로 훈련 세트에 더 많은 샘플을 사용할 수 있음.

- 이 OOB 점수를 평균하여 출력하기 위해 'RandomForestClassifier' 클래스의 'oob_score' 매개변수를 Ture로 지정해야 함. 기본값은 False. 

In [4]:
rf = RandomForestClassifier(oob_score = True, n_jobs = -1, random_state = 42)

rf.fit(train_input, train_target)
print(rf.oob_score_)
# 교차 검증에서 얻은 점수와 비슷한 결과를 얻음.

0.8934000384837406


## 엑스트라 트리

- '엑스트라 트리(Extra Trees)' 는 랜덤 포레스트와 매우 비슷하게 작동하는 클래스로, 기본적으로 100개의 결정 트리를 훈련하며, 랜덤 포레스트와 동일하게 결정 트리가 제공하는 매개변수 대부분을 지원함. 

- 일부 특성을 랜덤하게 선택하여 노드를 분할하는 데 사용함.

- 결정 트리를 만들 때 전체 훈련 세트를 사용하므로, 부트스트랩 샘플을 사용하지 않음.

- 노드를 분할할 때 가장 좋은 분할을 찾지 않고 무작위로 분할함. 이렇게 무작위로 분할하면 모델 성능이 낮아지겠지만, 많은 트리를 앙상블 하기 때문에 과대적합을 막고 검증 세트 점수를 높이는 효과가 있음. 아래에서 추가로 설명함.

- 'DecisionTreeClassifier' 클래스의 'splitter' 매개변수를 'random' 으로 지정하면 엑스트라 트리와 동일함.

- 'ExtraTreesClassifier' 클래스는 분류 모델, 'ExtraTreesRegressor' 클래스는 회귀 모델임.

### 교차 검증 수행하기

In [5]:
# 교차 검증을 수행함.
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs = -1, random_state = 42)

scores = cross_validate(et, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 랜덤 포레스트와 비슷한 결과를 출력함. 이 예제는 특성이 많지 않아서 두 모델의 차이가 크지 않음.
# 보통 엑스트라 트리가 무작위성이 더 크기 때문에 랜덤 포레스트보다 더 많은 결정 트리를 훈련해야 함.
# 더 많은 결정 트리를 훈련하지만 노드를 랜덤하게 분할하므로 계산 속도가 빠름.

0.9974503966084433 0.8887848893166506


### 모델 훈련 후 특성 중요도 출력하기

In [6]:
et.fit(train_input, train_target)
print(et.feature_importances_)
# 앞의 1절에서 결정 트리 모델의 특성 중요도는 [0.12345626, 0.86862934, 0.0079144] 이고, 각각 alcohol, sugar, pH를 나타냄.
# 엑스트라 트리의 점수와 비교하면 alcohol과 pH의 중요도가 상승하고 sugar의 중요도가 감소함. 

[0.20183568 0.52242907 0.27573525]


## 그레이디언트 부스팅

- '그레이디언트 부스팅(Gradient boosting)' 은 깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식으로 앙상블 하는 방법이며, 그레이디언트(Gradient)의 의미처럼 '경사 하강법' 을 사용하여 트리를 앙상블에 추가함. 이전에 설명했듯이 '경사 하강법' 은 손실 함수를 산으로 정의하고 가장 낮은 곳을 찾아 내려오는 과정임.

- 모델의 가중치와 절편을 조금씩 바꾸면서 가장 낮은 곳으로 내려옴. 손실 함수의 낮은 곳으로 천천히 조금씩 이동하기 위해 깊이가 얕은 결정 트리를 계속 추가하면서 이동하는 방법을 취함.

- 분류에서는 '로지스틱 손실 함수' 를 사용하고, 회귀에서는 '평균 제곱 오차 함수' 를 사용함.

- GradientBoostingClassifier: 기본적으로 깊이가 3인 결정 트리를 100개 사용하는 분류 클래스. 깊이가 얕은 결정 트리를 사용하므로 과대적합에 강하고 높은 일반화 성능을 낼 수 있음.

- GradientBoostingRegressor: 그레이디언트 부스팅의 회귀 클래스.

 - n_estimators: 트리 개수를 지정하는 매개변수. 기본값은 100.

 - learning_rate: 학습률을 지정하는 매개변수. 속도를 조절할 수 있으며 기본값은 0.1.

 - subsample: 트리 훈련에 사용하는 훈련 세트의 비율을 정하는 매개변수. 기본값은 1.0으로 전체 훈련 세트를 사용함. 1.0보다 작으면 훈련 세트 일부를 사용하며, 경사 하강법 단계마다 일부 샘플을 랜덤하게 선택하여 진행하는 '확률적 경사 하강법'이나 '미니배치 경사 하강법' 과 비슷함.

- 일반적으로 그레이디언트 부스팅이 랜덤 포레스트보다 조금 더 높은 성능을 내지만, 순서대로 트리를 추가하기 때문에 훈련 속도가 느림. 즉 'GradientBoostingClassifier' 클래스에는 'n_jobs' 매개변수가 없음.

### 교차 검증 수행하기

In [13]:
# 'GradientBoostingClassifier' 클래스를 사용하여 교차 검증 점수를 확인함.
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state = 42)

scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))
# 과대적합이 거의 되지 않음.
# 그레이디언트 부스팅은 결정 트리 개수를 늘려도 과대적합에 매우 강함.
# 학습률을 증가시키고 트리 개수를 늘리면 성능이 더 향상될 수 있음.

0.8881086892152563 0.8720430147331015


### 매개변수를 조정하여 점수 출력하기

In [14]:
# 'n_estimators' 매개변수 값을 변경하여 결정 트리 개수를 500개로 늘리고
# 'learning_rate' 매개변수 값을 변경하여 학습률을 0.2로 지정함.
gb = GradientBoostingClassifier(n_estimators = 500, learning_rate = 0.2, random_state = 42)

scores = cross_validate(gb, train_input, train_target, return_train_score = True, n_jobs = -1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


### 모델 훈련 후 특성 중요도 출력하기

In [15]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)
# 그레이디언트 부스팅은 sugar에 0.68010884만큼 집중하며,
# 랜덤 포레스트의 sugar에 대한 집중도(0.50039841)보다 더 비중을 두고 집중함.

[0.15872278 0.68010884 0.16116839]


## 히스토그램 기반 그레이디언트 부스팅